In [1]:
import random
import sys
sys.path.append('src/')
sys.path.append('experiments/')
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_curve, auc, mean_squared_error
from sklearn.model_selection import train_test_split

from src.clause_generator import ClauseGenerator
from src.ilp_solver import ILPSolver
from src.ilp_problem import ILPProblem
from src.data_utils import DataUtils
from src.visualize import Visualize
from experiments.parameters import get_parameters
from experiments.eval_utils import get_dataset_with_noise, compute_auc, compute_mse, extract

In [2]:
### problem
problem = 'member'
#problem = 'delete'
#problem = 'plus'
#problem = 'append'
#problem = 'subtree'

### noise_rate
noise_rate = 0.05

In [5]:
test_size = 0.3
du = DataUtils(problem)
pos, neg, bk, clauses, lang = du.load_data()
pos_train, pos_test = train_test_split(
    pos, test_size=test_size, random_state=7014)
neg_train, neg_test = train_test_split(
    neg, test_size=test_size, random_state=7014)

N_beam, T_beam, T, m = get_parameters(problem)

### add noise
pos_train_, neg_train_ = get_dataset_with_noise(pos_train, neg_train, noise_rate=noise_rate)
ilp_train = ILPProblem(pos_train_, neg_train_, bk, lang, name=problem)
ilp_train.print()

======= POSITIVE EXAMPLES =======
[member(c,f(c,f(a,f(c,f(a,*))))), member(a,f(a,f(c,*))), member(b,f(c,f(a,f(a,f(b,*))))), member(c,f(c,f(b,f(a,*)))), member(b,f(b,f(c,f(b,*)))), member(b,f(c,f(b,f(c,f(a,*))))), member(a,f(a,f(c,*))), member(c,f(c,f(b,f(c,f(c,*))))), member(c,f(b,f(a,f(c,f(a,*))))), member(c,f(a,f(c,f(c,f(a,*))))), member(c,f(c,f(a,*))), member(b,f(a,f(b,f(b,f(b,*))))), member(a,f(c,f(a,*))), member(a,f(a,f(c,f(c,*)))), member(b,f(c,f(c,f(b,*)))), member(b,f(b,f(c,*))), member(b,f(a,f(c,f(b,*)))), member(c,f(b,f(c,*))), member(a,f(b,f(a,f(b,*)))), member(b,f(b,f(c,*))), member(b,f(b,f(a,f(a,f(a,*))))), member(a,f(c,f(c,f(a,f(b,*))))), member(c,f(c,f(a,*))), member(b,f(b,f(c,f(c,*)))), member(b,f(a,f(a,f(b,f(b,*))))), member(a,f(b,f(a,f(c,*)))), member(a,f(a,f(c,f(b,f(b,*))))), member(c,f(c,f(c,*))), member(a,f(a,f(c,f(a,f(c,*))))), member(c,f(c,f(a,f(c,*)))), member(a,f(c,f(a,f(c,*)))), member(a,f(b,f(b,f(c,f(a,*))))), member(c,f(c,f(b,*))), member(a,f(c,f(b,f(a,f(b,*

In [6]:
CG = ClauseGenerator(ilp_train, infer_step=T, max_depth=1, max_body_len=1)
solver = ILPSolver(ilp_train, C_0=clauses, CG=CG, m=m, infer_step=T)

In [7]:
N_max = 50 # maximum number of clauses
lr = 1e-2
epoch = 2000

### train weights
clauses, Ws, loss_list = solver.train(
        gen_mode='beam', N_max=N_max, T_beam=T_beam, N_beam=N_beam, epoch=epoch, lr=lr, wd=0.0)

Generating clauses...
======= BEAM SEARCHED CLAUSES ======
member(X,X):-.
member(X,Y):-.
member(X,Y):-member(Y,X).
member(X,f(X,Y)):-.
member(X,f(Y,Z)):-.
member(X,f(Y,Z)):-member(X,Z).
member(X,f(Y,Z)):-member(Y,Z).
Enumerating facts...
Building tensors...


  1%|          | 11/2000 [00:00<00:18, 106.75it/s, loss=0.53] 

Learning weights...


100%|██████████| 2000/2000 [00:14<00:00, 135.73it/s, loss=0.0197]  

====== LEARNED PROGRAM ======
C_0:  member(X,f(X,Y)):-. 0.9619221091270447
C_1:  member(X,f(Y,Z)):-member(X,Z). 0.9805846214294434


In [8]:
### plot training loss
plt.plot(loss_list)
plt.show()

<ipython-input-8-6dca77e7924e>:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [9]:
### evaluate on test data
v_list, facts = solver.predict(pos_test, neg_test, clauses, Ws)
mse = compute_mse(pos_test, neg_test, v_list, facts)
auc = compute_auc(pos_test, neg_test, v_list, facts)

print('====== TEST SCORE =======')
print('Mean-squared test error: ', mse)
print('AUC: ', auc)

Enumerating facts...
Building tensors...
====== TEST SCORE =======
Mean-squared test error:  0.0007501581180717439
AUC:  1.0


In [23]:
# Visualize v_0, v_1, ..., v_n after learning

#v_list, facts = solver.get_valuation_memory(pos_test, neg_test, clauses, Ws)
#ilp_test = ILPProblem(pos_test, neg_test, bk, lang, name=name)
#vis = Visualize(ilp_test, name=name, facts=facts)
#vis.plot_valuation_memory(v_list)